# Here is my implementation for 2-class classiifiation problem

In [84]:
import pandas as pd
import numpy as np
import sklearn
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report

### Test with another data

In [85]:
trainData = pd.read_csv("https://raw.githubusercontent.com/Vasistareddy/sentiment_analysis/master/data/train.csv")
# test Data
testData = pd.read_csv("https://raw.githubusercontent.com/Vasistareddy/sentiment_analysis/master/data/test.csv")

In [86]:
trainData.head()

,Content,Label
0,every once in a while you see a film that is s...,pos
1,the love for family is one of the strongest dr...,pos
2,after the terminally bleak reservoir dogs and ...,pos
3,( warning to those who have not seen seven : ...,pos
4,"having not seen , "" who framed roger rabbit "" ...",pos


In [87]:
# Preprocessing
trainData['Content'] = [sen.lower() for sen in trainData['Content']]

mp = {'pos': 1, 'neg': -1}
trainData['Label'] = trainData['Label'].map(mp)

testData['Content'] = [sen.lower() for sen in testData['Content']]

mp = {'pos': 1, 'neg': -1}
testData['Label'] = testData['Label'].map(mp)

In [88]:
trainData.dropna(inplace=True) #removing all empty spaces
testData.dropna(inplace=True) #removing all empty spaces

In [89]:
sentences = [sen.split() for sen in trainData["Content"]]
print(sentences[0]) # like word_tokenize

sen2s = [sen.split() for sen in testData['Content']]

['every', 'once', 'in', 'a', 'while', 'you', 'see', 'a', 'film', 'that', 'is', 'so', 'effective', 'in', 'delivering', 'the', 'goods', 'that', 'it', 'is', 'easy', 'to', 'forget', ',', 'and', 'forgive', ',', 'its', 'glaring', 'imperfections', '.', 'such', 'is', 'the', 'case', 'with', '?', 'good', 'will', 'hunting', '?', ',', 'a', 'subtle', 'character', 'study', 'about', 'a', 'socially', 'inept', 'mathematics', 'genius', 'who', 'struggles', 'to', 'find', 'his', 'path', 'in', 'life', '.', 'despite', 'some', 'serious', 'character', 'problems', ',', 'this', 'is', 'still', 'a', 'very', 'good', 'film', '.', 'you', 'probably', 'know', 'about', 'the', 'plot', 'so', 'i', '?', 'll', 'make', 'it', 'quick', '.', 'will', 'hunting', '(', 'damon', ')', 'is', 'a', 'janitor', 'at', 'm', '.', 'i', '.', 't', '.', ',', 'he', '?', 's', 'really', 'smarter', 'than', 'einstein', 'but', 'nobody', 'knows', 'it', '.', 'he', 'likes', 'to', 'go', 'out', 'with', 'his', 'friend', 'chuckie', '(', 'affleck', ')', 'and',

In [90]:
# punc
for i in range(len(sentences)):
    sentences[i] = [word for word in sentences[i] if word.isalnum()]
print(sentences[0])

# punc
for i in range(len(sen2s)):
    sen2s[i] = [word for word in sen2s[i] if word.isalnum()]
print(sen2s[0])

['every', 'once', 'in', 'a', 'while', 'you', 'see', 'a', 'film', 'that', 'is', 'so', 'effective', 'in', 'delivering', 'the', 'goods', 'that', 'it', 'is', 'easy', 'to', 'forget', 'and', 'forgive', 'its', 'glaring', 'imperfections', 'such', 'is', 'the', 'case', 'with', 'good', 'will', 'hunting', 'a', 'subtle', 'character', 'study', 'about', 'a', 'socially', 'inept', 'mathematics', 'genius', 'who', 'struggles', 'to', 'find', 'his', 'path', 'in', 'life', 'despite', 'some', 'serious', 'character', 'problems', 'this', 'is', 'still', 'a', 'very', 'good', 'film', 'you', 'probably', 'know', 'about', 'the', 'plot', 'so', 'i', 'll', 'make', 'it', 'quick', 'will', 'hunting', 'damon', 'is', 'a', 'janitor', 'at', 'm', 'i', 't', 'he', 's', 'really', 'smarter', 'than', 'einstein', 'but', 'nobody', 'knows', 'it', 'he', 'likes', 'to', 'go', 'out', 'with', 'his', 'friend', 'chuckie', 'affleck', 'and', 'their', 'other', 'buddies', 'and', 'drink', 'beer', 'he', 's', 'charismatic', 'and', 'witty', 'but', 'h

In [91]:
# stopWords
stop_words = set(stopwords.words('english'))

for i in range(len(sentences)):
    sentences[i] = [word for word in sentences[i] if word not in stop_words]
print(sentences[0])
# Output: don't have stop words in sentence[i] any 


for i in range(len(sen2s)):
    sen2s[i] = [word for word in sen2s[i] if word not in stop_words]
print(sen2s[0])
# Output: don't have stop words in sentence[i] any more

['every', 'see', 'film', 'effective', 'delivering', 'goods', 'easy', 'forget', 'forgive', 'glaring', 'imperfections', 'case', 'good', 'hunting', 'subtle', 'character', 'study', 'socially', 'inept', 'mathematics', 'genius', 'struggles', 'find', 'path', 'life', 'despite', 'serious', 'character', 'problems', 'still', 'good', 'film', 'probably', 'know', 'plot', 'make', 'quick', 'hunting', 'damon', 'janitor', 'really', 'smarter', 'einstein', 'nobody', 'knows', 'likes', 'go', 'friend', 'chuckie', 'affleck', 'buddies', 'drink', 'beer', 'charismatic', 'witty', 'terrible', 'time', 'authority', 'stiff', 'college', 'folk', 'getting', 'tiff', 'law', 'distinguished', 'professor', 'skarsg', 'rd', 'discovers', 'genius', 'offers', 'chance', 'clean', 'record', 'avoid', 'jail', 'time', 'price', 'must', 'attend', 'weekly', 'sessions', 'therapist', 'work', 'various', 'mathematical', 'problems', 'stumped', 'academic', 'elite', 'professor', 'outsmarting', 'scaring', 'hell', 'couple', 'different', 'psycholog

In [92]:
stemmer = PorterStemmer()

for i in range(len(sentences)):
    sentences[i] = [stemmer.stem(word) for word in sentences[i]]

print(sentences[0])
# Output: running to run, location to locat

for i in range(len(sen2s)):
    sen2s[i] = [stemmer.stem(word) for word in sen2s[i]]

print(sen2s[0])
# Output: running to run, location to locat

['everi', 'see', 'film', 'effect', 'deliv', 'good', 'easi', 'forget', 'forgiv', 'glare', 'imperfect', 'case', 'good', 'hunt', 'subtl', 'charact', 'studi', 'social', 'inept', 'mathemat', 'geniu', 'struggl', 'find', 'path', 'life', 'despit', 'seriou', 'charact', 'problem', 'still', 'good', 'film', 'probabl', 'know', 'plot', 'make', 'quick', 'hunt', 'damon', 'janitor', 'realli', 'smarter', 'einstein', 'nobodi', 'know', 'like', 'go', 'friend', 'chucki', 'affleck', 'buddi', 'drink', 'beer', 'charismat', 'witti', 'terribl', 'time', 'author', 'stiff', 'colleg', 'folk', 'get', 'tiff', 'law', 'distinguish', 'professor', 'skarsg', 'rd', 'discov', 'geniu', 'offer', 'chanc', 'clean', 'record', 'avoid', 'jail', 'time', 'price', 'must', 'attend', 'weekli', 'session', 'therapist', 'work', 'variou', 'mathemat', 'problem', 'stump', 'academ', 'elit', 'professor', 'outsmart', 'scare', 'hell', 'coupl', 'differ', 'psychologist', 'meet', 'match', 'hook', 'therapist', 'name', 'sean', 'maguir', 'william', 'pr

In [93]:
X_train = [' '.join(sentence) for sentence in sentences]
X_test = [' '.join(sen) for sen in sen2s]

# Target variable
y_train = trainData['Label'].values 
y_test = testData['Label'].values

# Vectorize the text data using TfidfVectorizer
tfidf = TfidfVectorizer(max_features=15000)
X_train_tfidf = tfidf.fit_transform(X_train)
X_test_tfidf = tfidf.transform(X_test)

print(X_train_tfidf.toarray())


[[0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.03794973 0.         ... 0.         0.         0.        ]
 ...
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]]


In [94]:
def convert_to_dense_if_sparse(X):
    if hasattr(X, 'toarray'):
        return X.toarray()
    return X

class SVM_classifier():
    def __init__(self, learning_rate=0.01, lambda_=10, iterations=1000):
        self.learning_rate = learning_rate
        self.lambda_ = lambda_
        self.iterations = iterations
        self.W = None
        self.B = None
    
    def compute_hingle_loss(self, W, B, X_batch, y_batch):
        """ calculate hinge loss """
        N = X_batch.shape[0]
        distance = []
        for idx, x in enumerate(X_batch):
            distance.append(max(0, 1 - y_batch[idx] * (np.dot(x, W) + B)))

        distances = np.array(distance) # let distance into numpy array 
    
        hinge_loss = self.lambda_ * (np.sum(distances) / N) # find hinge loss
        
        # calculate cost
        cost = 1 / 2 * np.dot(W, W) + hinge_loss
        return cost

    def gradientDescent(self, W, B, X_batch, Y_batch):
        distance = []
        for idx, x in enumerate(X_batch):
            distance.append(1 - Y_batch[idx] * (np.dot(x, W) + B))
    
        dw = np.zeros(len(W))
        dB = 0
        for idx, d in enumerate(distance):
            if max(0, d) == 0:
                dw += W
                dB += 0
            else:
                dw += W - (self.lambda_ * Y_batch[idx] * X_batch[idx])
                dB += 0 - (self.lambda_ * Y_batch[idx])
        
        dw = dw / len(Y_batch)  # average
        dB = dB / len(Y_batch)  # avg
        return dw, dB
        
    def fit(self, features, outputs) -> bool:
        # print(features)
        features = convert_to_dense_if_sparse(features)

        # print(features.shape)
        max_epochs = self.iterations
        weights = np.zeros(features.shape[1])
        bias = 0
        nth = 0

        prev_cost = float("inf")
        cost_threshold = 0.01  # in percent
        
        for epoch in range(1, max_epochs):
            gradW, gradB = self.gradientDescent(weights, bias, features, outputs)

            # convergence check on 2^nth epoch
            if epoch == 2 ** nth or epoch == max_epochs - 1:
                cost = self.compute_hingle_loss(weights, bias, features, outputs)
                print("Epoch is:{} and Cost is: {}".format(epoch, cost))
                # stoppage criterion
                if abs(prev_cost - cost) < cost_threshold * prev_cost:
                    self.W = weights
                    self.B = bias
                    return True
                prev_cost = cost
                nth += 1
            
            # update grad
            weights = weights - (self.learning_rate * gradW)
            bias = bias - (self.learning_rate * gradB)
            
        self.W = weights
        self.B = bias
        return True
    
    def decisionFunc(self, X):
        X = convert_to_dense_if_sparse(X)
        ans = []
        for x in X:
            ans.append(np.dot(x, self.W) + self.B)
        return np.array(ans)
    
    def predict(self, X):
        X = convert_to_dense_if_sparse(X)
        # print(X)
        prediction = []
        for x in X:
            prediction.append(np.dot(x, self.W) + self.B) # w.x + b
        
        # print(np.sign(prediction))
        return np.sign(prediction)

    # Evaluate the model
    def evaluate(self, X_test, y_test):
        predictions = self.predict(X_test)
        correct = 0
        cnt_pos = 0
        cnt_neg = 0
        for i in range(predictions.shape[0]):
            if predictions[i] == y_test[i]:
                correct += 1
            
            if y_test[i] == 1:
                cnt_pos += 1
            else:
                cnt_neg += 1
        accuracy = correct / y_test.shape[0]
        print(f'Accuracy: {accuracy * 100:.2f}%')
        print(f"Pos_rate: {cnt_pos / y_test.shape[0] * 100:.2}%")
        return accuracy

In [95]:
class OvRClassifier:
    def __init__(self, n_classes, learning_rate=0.001, lambda_=10000, iterations=1000):
        self.n_classes = n_classes
        self.models = [SVM_classifier(learning_rate, lambda_, iterations) for _ in range(n_classes)]

    def fit(self, X, y):
        for i in range(self.n_classes):
            y_binary = np.where(y == i, 1, -1) # in y and val == i then in y_binary = 1, others = -1 
            self.models[i].fit(X, y_binary)

    def predict(self, X):
        decision_values = np.array([model.decisionFunc(X) for model in self.models])
        return np.argmax(decision_values, axis=0)
    
    def evaluate(self, X_test, y_test):
        predictions = self.predict(X_test)
        correct = 0
        cnt_pos = 0
        cnt_neg = 0
        cnt_net = 0
        for i in range(predictions.shape[0]):
            if predictions[i] == y_test[i]:
                correct += 1
            
            if y_test[i] == 2:
                cnt_pos += 1
            elif y_test[i] == 0:
                cnt_neg += 1
            else:
                cnt_net += 1
                
        accuracy = correct / y_test.shape[0]
        print(f'Accuracy: {accuracy * 100:.2f}%')
        print(f"Number: {cnt_pos}, {cnt_neg}, {cnt_net}")
        return accuracy

In [96]:
svm = SVM_classifier(learning_rate=0.01, lambda_=100)
svm.fit(X_train_tfidf, y_train)

Epoch is:1 and Cost is: 100.0
Epoch is:2 and Cost is: 99.84315793374063


True

In [97]:
svm.evaluate(X_train_tfidf, y_train)
svm.evaluate(X_test_tfidf, y_test)

Accuracy: 89.39%
Pos_rate: 5e+01%
Accuracy: 83.50%
Pos_rate: 5e+01%


0.835

In [102]:
s = SVC(kernel='linear', random_state=42)
s.fit(X_train_tfidf, y_train)

y_pred = s.predict(X_train_tfidf)

# Evaluate the model
accuracy = accuracy_score(y_pred, y_train)
print(f"Accuracy: {accuracy * 100:.2f}%")
# print("\nClassification Report:")
# print(classification_report(y_test, y_pred))

Accuracy: 99.06%


In [103]:
predictions = s.predict(X_test_tfidf)
cnt_true = 0
n = predictions.shape[0]
for i in range(n):
    if predictions[i] == y_test[i]:
        cnt_true += 1
accuracy = cnt_true / n * 100
print(round(accuracy, 2), "%", sep="")

86.0%
